In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
from datetime import datetime
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  #显示中文
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
import pylab
pylab.rcParams['figure.figsize'] = (15.0, 8.0) # 显示大小

In [2]:
###检查文件是否符合规范

In [3]:
def hand_col(df,col ="发动机转速",n=240):
    
    ####计算col停顿时间
    df["last_notna_time"] = df[df[col].notna()].Time   ##当时不是na的时间
    df_notna_time = df[df[col].notna()].Time
    df_notna_time_diff = (df_notna_time - df_notna_time.shift(1)).dt.total_seconds()
    df["last_notna_time_diff"] = df_notna_time_diff
    df["last_notna_time_diff"] = df["last_notna_time_diff"].fillna(method = "bfill")
    
    df.loc[df["last_notna_time_diff"]> n,col] = df.loc[df["last_notna_time_diff"]> n,col].fillna(0)  #长时间na的填充0
    df[col] = df[col].fillna(method = "ffill")
    return df

In [4]:
def cz(df,idx,col="累计油耗"):
    num = len(idx)
    for i in range(num-1):
        start = idx.iloc[i]
        end = idx.iloc[i+1]
        cz_num = end-start
        start_num = df.loc[start,col]
        end_num = df.loc[end,col]
        fillna = np.linspace(start_num,end_num,cz_num+1)
        df.loc[start:end,col] = fillna
    return df

In [5]:
def hand_col_cz(df,col ="累计油耗"):   ###插值处理
    
    ####计算col停顿间隔
    df["last_notna_idx"] = df[df[col].notna()].idx   ##当时不是na的idx
    df_notna_idx = df[df[col].notna()].idx
    df_notna_idx_diff = (df_notna_idx - df_notna_idx.shift(1))
    df["last_notna_idx_diff"] = df_notna_idx_diff
    df["last_notna_idx_diff"] = df["last_notna_idx_diff"].fillna(method = "bfill")
    
    df.loc[df["last_notna_time_diff"]> n,col] = df.loc[df["last_notna_time_diff"]> n,col].fillna(0)  #长时间na的填充0
    df[col] = df[col].fillna(method = "ffill")
    return df

In [6]:
def hand(df):
    df = df.drop(["GPS纬度","GPS经度","GPS定位时间"],axis=1)#去除无用数据
    df["Time"] = pd.to_datetime(df["time"].str[:-3])#时间转化,直接去掉小数部分
    
    ####停机以发动机数据为准,处理缺失发动机转数
    df = hand_col(df,col ="发动机转速",n=240)
    
    df = hand_col(df,col ="系统压力",n=240)
    
    df = hand_col(df,col ="转向次数",n=240)
    
    df = hand_col(df,col ="泵送排量",n=240)
    
    df = hand_col(df,col ="泵车状态",n=120)
    
    ####油耗单独填充,插值
    df_notna_idx = df[df["累计油耗"].notna()].idx
    df = cz(df,df_notna_idx,col="累计油耗")
    ###
    
    
    ####泵送计时填充
    df["累计时间"] = df["累计时间"].fillna(method = "ffill")
    
    
    
    ####液压油温用插值
    df_notna_idx = df[df["液压油温"].notna()].idx
    df = cz(df,df_notna_idx,col="液压油温")
    return df

In [22]:
def check(f):
    df = pd.read_csv(f,skiprows=1)
    df["Time"] = pd.to_datetime(df["time"].str[:-3])#时间转化,直接去掉小数部分
    df["idx"] = np.arange(len(df))
    df = hand(df)
    df = df.dropna()
    df["second"] = df.Time.apply(lambda x: time.mktime(x.timetuple()))
    second_start = df.iloc[0].second
    df["five_second"] = (df.second - second_start)  // 5
    df_mean = df.drop("time",axis=1).groupby("five_second").mean()
    df_mean["泵车状态"] = df[["泵车状态","five_second"]].groupby("five_second").max()
    start = df.iloc[0].second
    end = df.iloc[-1].second
    second_index = np.arange(start,end,5)
    
    second_idx = pd.DataFrame({"second_idx":second_index})
    lenth  = len(second_idx)/12/60/24  ###数据持续时间 天  2~3天正常
    
    new_df = second_idx.join(df_mean)
    new_df = new_df.drop(["idx","last_notna_time_diff","second"],axis=1)
    new_df["time"] = pd.to_datetime(new_df['second_idx'].values, unit='s',utc=True).tz_convert("Asia/Shanghai").tz_localize(None)
    fdj_time = len(new_df[new_df["发动机转速"]>0])/12/60   ###发动机累积工作时间  5~6小时正常，太短则可能遇到数据传输缺失的问题
    
    bs1 = len(new_df[new_df["泵车状态"] == 3])
    bs2 = len(new_df[new_df["转向次数"] >0 ])
    
    if lenth>=2 and fdj_time >=5 and bs1>15*12*0.7 and bs2>15*12*0.7:
        return True
    else:
        return False

In [23]:
import os

In [24]:
dic = {}
for f in os.listdir("xxxxd"):
    dic[f]=check(f)

OSError: Initializing from file failed

In [7]:
df = pd.read_csv("data.csv",skiprows=1)
df["Time"] = pd.to_datetime(df["time"].str[:-3])#时间转化,直接去掉小数部分
df["idx"] = np.arange(len(df))

In [8]:
df = hand(df)

In [9]:
df = df.dropna()

In [10]:
df["second"] = df.Time.apply(lambda x: time.mktime(x.timetuple()))
second_start = df.iloc[0].second
df["five_second"] = (df.second - second_start)  // 5

In [11]:
df_mean = df.drop("time",axis=1).groupby("five_second").mean()

In [12]:
df_mean["泵车状态"] = df[["泵车状态","five_second"]].groupby("five_second").max()

In [13]:
start = df.iloc[0].second
end = df.iloc[-1].second
second_index = np.arange(start,end,5)

In [16]:
second_idx = pd.DataFrame({"second_idx":second_index})
len(second_idx)/12/60/24  ###数据持续时间 天  2~3天正常
new_df = second_idx.join(df_mean)
new_df = new_df.drop(["idx","last_notna_time_diff","second"],axis=1)
new_df["time"] = pd.to_datetime(new_df['second_idx'].values, unit='s',utc=True).tz_convert("Asia/Shanghai").tz_localize(None)


In [17]:
len(second_idx)/12/60/24  ###数据持续时间 天  2~3天正常

2.628645833333333

In [15]:
len(new_df[new_df["发动机转速"]>0])/12/60   ###发动机累积工作时间  5~6小时正常，太短则可能遇到数据传输缺失的问题

6.673611111111112

In [19]:
len(new_df[new_df["泵车状态"] == 3])

652

In [20]:
len(new_df[new_df["转向次数"] >0 ])

1184

In [18]:
len(new_df[new_df["泵车状态"] == 3])>15*12*0.7
len(new_df[new_df["转向次数"] >0 ])> 15*12*0.7

,second_idx,液压油温,泵送排量,累计油耗,转向次数,发动机转速,系统压力,累计时间,泵车状态,time
0,1.564249e+09,50.0,0.0,2.050000,0.0,559.00,0.0,0.00,0.0,2019-07-28 01:31:59
1,1.564249e+09,50.0,0.0,2.133333,0.0,550.60,0.0,0.00,0.0,2019-07-28 01:32:04
2,1.564249e+09,50.0,0.0,2.212500,0.0,560.50,0.0,0.00,0.0,2019-07-28 01:32:09
3,1.564249e+09,50.0,0.0,2.308333,0.0,563.75,0.0,0.00,0.0,2019-07-28 01:32:14
4,1.564249e+09,50.0,0.0,2.383333,0.0,569.80,0.0,0.00,0.0,2019-07-28 01:32:19
5,1.564249e+09,50.0,0.0,2.466667,0.0,556.00,0.0,0.00,0.0,2019-07-28 01:32:24
6,1.564249e+09,50.0,0.0,2.541667,0.0,561.25,0.0,0.00,0.0,2019-07-28 01:32:29
7,1.564249e+09,50.0,0.0,2.608333,0.0,559.75,0.0,0.00,0.0,2019-07-28 01:32:34
8,1.564249e+09,50.0,0.0,2.700000,0.0,555.80,0.0,0.00,0.0,2019-07-28 01:32:39
9,1.564249e+09,50.0,0.0,2.783333,0.0,559.80,0.0,0.00,0.0,2019-07-28 01:32:44
